In [37]:
import fiftyone as fo

# Delete the existing dataset
if fo.dataset_exists("circle-pond-2"):
    fo.delete_dataset("circle-pond-2")

# Now create your new dataset
dataset = fo.Dataset("circle-pond-2")

In [40]:
import fiftyone as fo
import os
import glob

# Set the dataset name
name = "circle-pond-2"

# Set the dataset directory
dataset_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\detection drone\runs-detections-drone-14.08\circle-pond-2"

# The splits to load
splits = ["test", "val"]

# Create or load the dataset
if fo.dataset_exists(name):
    dataset = fo.load_dataset(name)
    print(f"Loaded existing dataset '{name}'")
else:
    dataset = fo.Dataset(name)
    print(f"Created new dataset '{name}'")

def read_yolo_label(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    detections = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        detections.append(
            fo.Detection(
                label=str(int(class_id)),  # Convert class_id to string
                bounding_box=[x_center - width/2, y_center - height/2, width, height]
            )
        )
    return detections

for split in splits:
    split_dir = os.path.join(dataset_dir, split)
    images_dir = os.path.join(split_dir, "images")
    labels_dir = os.path.join(split_dir, "labels")
    
    # Add images and labels
    for image_path in glob.glob(os.path.join(images_dir, "*")):
        image_name = os.path.basename(image_path)
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_name)
        
        sample = fo.Sample(filepath=image_path)
        sample.tags.append(split)
        
        if os.path.exists(label_path):
            detections = read_yolo_label(label_path)
            sample["ground_truth"] = fo.Detections(detections=detections)
        
        dataset.add_sample(sample)
        print(f"Added {image_name} with label (if exists)")

print(f"\nDataset '{name}' now has {len(dataset)} samples")

# Print detailed information about the dataset
print("\nDataset Summary:")
print(dataset.summary())

# Print schema of the dataset
print("\nDataset Schema:")
for field_name, field_type in dataset.get_field_schema().items():
    print(f"{field_name}: {field_type}")

# If you want to examine a specific sample
if len(dataset) > 0:
    sample = dataset.first()
    print("\nFirst Sample Details:")
    print(sample)

    if "ground_truth" in sample:
        print("\nGround Truth for First Sample:")
        print(sample.ground_truth)
    else:
        print("\nWarning: 'ground_truth' field not found in the first sample")
else:
    print("\nWarning: Dataset is empty")

# Optionally, you can visualize the dataset
# session = fo.launch_app(dataset)
# session.wait()


Loaded existing dataset 'circle-pond-2'
Added 20230920_121217_jpg.rf.555850596373d9093b5263e90fd0b0a7.jpg with label (if exists)
Added 20230920_121220_jpg.rf.ac37e69c72ff509e032d4e378189c9d1.jpg with label (if exists)
Added 20230920_121227_jpg.rf.d195c6e54f8d06a6f51505b978333c1c.jpg with label (if exists)
Added 20230920_121238_jpg.rf.c0192c9518e2c9e4efd18cade234e6e7.jpg with label (if exists)
Added 20230920_121250_jpg.rf.6bd25d544ea9f9867558c05f3ba88eb1.jpg with label (if exists)
Added 20230920_121254_jpg.rf.c16222fb0f320908468985d74cec3ec8.jpg with label (if exists)
Added 20230920_121259_jpg.rf.52f13a58daa39b915c4011a697910bfc.jpg with label (if exists)
Added 20230920_121318_jpg.rf.f94ae454fda993a57443d666c7e4e4d0.jpg with label (if exists)
Added 20230920_121322_jpg.rf.2781af8d8a0b210c47cbfd0e41ce84df.jpg with label (if exists)
Added 20230920_121325_jpg.rf.273c044db976c007f0ba375f14025c67.jpg with label (if exists)
Added 20230920_121331_jpg.rf.6aae391459311c230ad9a2979f627cc4.jpg with

In [42]:
session = fo.launch_app(dataset)

In [43]:
from ultralytics import RTDETR


model= RTDETR(r"C:\Users\gbo10\OneDrive\measurement_paper_images\detection drone\runs-detections-drone-14.08\detect\train\weights\best.pt")



dataset.view().apply_model(model, label_field="RTDETR")



 100% |███████████████████| 75/75 [1.0m elapsed, 0s remaining, 1.2 samples/s]      


In [49]:
import fiftyone as fo
import os

# Load your FiftyOne dataset
dataset_name = "your-dataset-name"
dataset = fo.load_dataset(dataset_name)

# Define the mapping between local folders and their corresponding tags
path_to_tag_map = {
    r"C:\Users\gbo10\Dropbox\research videos\before january 2024\underwater drone/20.9\natural": "natural",
    r"C:\Users\gbo10\Dropbox\research videos\before january 2024\underwater drone/20.9\light plus flash": "light+", 
         r"C:\Users\gbo10\Dropbox\research videos\before january 2024\underwater dron/20.9\with dark": "dark",  # Add more mappings as needed
 # Add more mappings as needed
}

# Create a dictionary to map base filenames to tags
filename_to_tag = {}

for folder_path, tag in path_to_tag_map.items():
    for root, _, files in os.walk(folder_path):
        for filename in files:
            # Extract the base filename (without extension)
            base_filename = os.path.splitext(filename)[0]
            filename_to_tag[base_filename] = tag

# Iterate through each sample in the FiftyOne dataset
for sample in dataset:
    # Extract the base filename from the sample's filepath
    sample_filename = os.path.basename(sample.filepath)

    # Find the common part of the filename (e.g., "20230920_121147")
    common_part = "_".join(sample_filename.split("_")[:2])  # Adjust the splitting logic if needed

    # Check if the common part exists in our filename_to_tag map
    if common_part in filename_to_tag:
        # Get the corresponding tag
        tag = filename_to_tag[common_part]

        # Append the tag to the sample and save it
        sample.tags.append(tag)
        sample.save()
        print(f"Tagged {sample.filepath} with '{tag}'")

print("Tagging complete!")


ConnectionError: HTTPConnectionPool(host='localhost', port=5151): Max retries exceeded with url: /event (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002322FC90C10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 s

In [45]:
import fiftyone as fo


print(fo.config)




{
    "batcher_static_size": 100,
    "batcher_target_latency": 0.2,
    "batcher_target_size_bytes": 1048576,
    "database_admin": true,
    "database_dir": "C:\\Users\\gbo10\\.fiftyone\\var\\lib\\mongo",
    "database_name": "fiftyone",
    "database_uri": null,
    "database_validation": true,
    "dataset_zoo_dir": "C:\\Users\\gbo10\\fiftyone",
    "dataset_zoo_manifest_paths": null,
    "default_app_address": "localhost",
    "default_app_port": 5151,
    "default_batch_size": null,
    "default_batcher": "latency",
    "default_dataset_dir": "C:\\Users\\gbo10\\fiftyone",
    "default_image_ext": ".jpg",
    "default_ml_backend": "torch",
    "default_sequence_idx": "%06d",
    "default_video_ext": ".mp4",
    "desktop_app": false,
    "do_not_track": false,
    "logging_level": "INFO",
    "max_process_pool_workers": null,
    "max_thread_pool_workers": null,
    "model_zoo_dir": "C:\\Users\\gbo10\\fiftyone\\__models__",
    "model_zoo_manifest_paths": null,
    "module_path": n


Could not connect session, trying again in 10 seconds

Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



In [ ]:
sample = dataset.first()
print(sample.ground_truth.detections[0])